In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [6]:
"""
Imports

"""
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Button, HTML, HBox, Dropdown, VBox, Label, ToggleButtons, Layout
import plotly.express as px
from IPython.display import clear_output
from io import StringIO
import qgrid
import plotly.graph_objects as go
import fileupload

In [7]:
parameter_info = HTML(
    '<br><div class="alert alert-block alert-success" align="left"><strong> Enter input parameters: <br><strong> </div>'
)


def generate_line(button):
    if line_options.children[1].value is not None and line_options.children[
            3].value is not None:
        updated_df = working_grid.get_changed_df()
        linechart = go.FigureWidget(
            px.line(updated_df,
                    x=line_options.children[1].value,
                    y=line_options.children[3].value))
        if len(viz_master_box.children) > 3:
            viz_master_box.children = viz_master_box.children[:-1] + tuple(
                [linechart])
        else:
            viz_master_box.children = viz_master_box.children + tuple(
                [linechart])


def plot_graph_master(button):
    # Line Chart
    global line_options
    line_options = HBox([
        Label('Select Dimension column:',
              layout=Layout(margin='5px 10px 0px 0')),
        Dropdown(options=working_grid.get_changed_df().columns,
                 layout=Layout(width='160px')),
        Label('Select Metric column:',
              layout=Layout(margin='5px 10px 0px 50px')),
        Dropdown(options=working_grid.get_changed_df().columns,
                 layout=Layout(width='160px')),
        Button(description='Generate!',
               button_style='success',
               layout=Layout(margin='5px 10px 0px 150px'))
    ])
    if button.new == 'Line':
        viz_master_box.children = tuple([viz_master_box.children[0]]) + tuple(
            [parameter_info]) + tuple([line_options])
        # line_options.children[1].observe(generate_line, names='value')
        # line_options.children[3].observe(generate_line, names='value')
        line_options.children[4].on_click(generate_line)

In [56]:
introduction = HTML(
    '<div class="alert alert-block alert-info" align="left"><strong>Visualization Module - Upload Custom .csv or\
    .txt files and Visualize them !<strong></div>')

chart_selection_toggle = ToggleButtons(options=['Line', 'Multi-Line', 'Bar'],
                                       value=None,
                                       button_style='info')
chart_selection_display = HBox(
    [Label('Select Visualization Type:'), chart_selection_toggle])

chart_selection_toggle.observe(plot_graph_master, names=['value'])


def file_uploaded(button):
    if 'csv' not in button['owner'].filename and 'txt' not in button[
            'owner'].filename:
        start_master()
        file_upload_hbox.children[2].value = 'Upload csv/txt only!'
    else:
        start_master()
        file_upload_button.disabled = True
        success_info = HTML(
            '<br><div class="alert alert-block alert-success" align="left"><strong>File Uploaded Successfully! \
            You can edit your data using the table below- <br><strong> </div>')

        start_info = HTML(
            '<br><div class="alert alert-block alert-success" align="left"><strong>Start Visualizing your Data below:\
            <br><strong> </div>')

        display(success_info)
        decoded = button['owner'].data.decode('utf-8')
        filename = button['owner'].filename
        file_upload_hbox.children[2].value = '\t{} ({:.2f} kB)'.format(
            filename,
            len(StringIO(decoded).read()) / 2**10)
        global working_grid
        working_dataframe = pd.read_csv(StringIO(decoded))
        working_dataframe['Row Number'] = working_dataframe.index + 1
        working_dataframe = working_dataframe.set_index('Row Number')
        working_grid = qgrid.show_grid(working_dataframe,
                                       grid_options={'forceFitColumns': False})
        display(working_grid)
        display(start_info)
        global viz_master_box
        viz_master_box = VBox([chart_selection_display])
        chart_selection_toggle.value = None
        display(viz_master_box)


def start_master():
    clear_output()
    display(introduction)
    global file_upload_hbox
    file_upload_button = fileupload.FileUploadWidget()
    file_upload_hbox = HBox([
        HTML('<strong>Select File:', layout=Layout(margin='2px 15px 0px 0px')),
        file_upload_button,
        Label('', layout=Layout(margin='2px 0px 0px 10px'))
    ])
    file_upload_button.observe(file_uploaded, names='data')
    display(file_upload_hbox)


start_master()

HTML(value='<div class="alert alert-block alert-info" align="left"><strong>Visualization Module - Upload Custo…

HTML(value='<br><div class="alert alert-block alert-success" align="left"><strong>File Uploaded Successfully! …

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

HTML(value='<br><div class="alert alert-block alert-success" align="left"><strong>Start Visualizing your Data …